***

* [Outline](../0_Introduction/0_introduction.ipynb)
* [Glossary](../0_Introduction/1_glossary.ipynb)
* [2. Mathematical Groundwork](2_0_introduction.ipynb)
    * Previous: [2.7 Fourier Theorems](2_7_fourier_theorems.ipynb)
    * Next: [2.9 Sampling Theory](2_9_sampling_theory.ipynb)

***

Import standard modules:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import HTML 
HTML('../style/course.css') #apply general CSS

Import section specific modules:

In [ ]:
from IPython.display import HTML
from ipywidgets import interact
HTML('../style/code_toggle.html')

## 2.8. The Discrete Fourier Transform (DFT) and the Fast Fourier Transform (FFT)<a id='math:sec:the_discrete_fourier_transform_and_the_fast_fourier_transform'></a>

The continuous version of the Fourier transform can only be computed when the integrals involved can be evaluated analytically, something which is not always possible in real life applications. This is true for a number of reasons, the most relevant of which are:

傅里叶变换的连续形式只有在涉及到积分的时候才能计算出来，而这在实际应用中并不总是可能的。这是有很多原因的，其中最相关的是

   * We don't always have the parametrisation of the signal that we want to find the Fourier transform of我们并不总能得到我们想要的参数化傅里叶变换的信号.
    
   * Signals are measured and recorded at a finite number of points信号在有限数量的点上被测量和记录.
    
   * Measured signals are contaminated by noise被测信号受噪声污染.

In such cases the discrete equivalent of the Fourier transform, called the discrete Fourier transform (DFT), is very useful. In fact, where the scale of the problem necessitates using a computer to perform calculations, the Fourier transform can only be implemented as the discrete equivalent. There are some subtleties we should be aware of when implementing the DFT. These mainly arise because it is very difficult to capture the full information present in a continuous signal with a finite number of samples. In this chapter we review the DFT and extend some of the most useful identities derived in the previous sections to the case where we only have acces to a finite number of samples. The subtleties that arise due to limited sampling will be discussed in the next section.

在这种情况下，傅里叶变换的离散等价，称为离散傅里叶变换(DFT)，是非常有用的。事实上，当问题的规模需要使用计算机进行计算时，傅里叶变换只能被实现为离散等价。在实现DFT时，我们应该注意一些细微之处。这主要是因为在有限样本的连续信号中很难捕捉到全部信息。在本章中，我们将回顾DFT并扩展一些最有用的恒等式

### 2.8.1 The discrete time Fourier transform (DTFT): definition<a id='math:sec:the_discrete_time_fourier_transform_definition'></a>

We start by introducing the discrete time Fourier transform (DTFT). The DTFT of a set $\left\{y_n \in \mathbb{C}\right\}_{n ~ \in ~ \mathbb{Z}}$ results in a Fourier series (see [$\S$ 2.3 &#10142;](../2_Mathematical_Groundwork/2_3_fourier_series.ipynb)) of the form

首先介绍离散时间傅里叶变换(DTFT)。DTFT的一组傅里叶级数$\left\{y_n \in \mathbb{C}\right\}_{n ~ \in ~ \mathbb{Z}}$计算结果(见2.3)

<a id='math:eq:8_001'></a><!--\label{math:eq:8_001}-->$$
Y_{2\pi}(\omega) = \sum_{n\,=\,-\infty}^{\infty} y_n\,e^{-\imath \omega n} \quad \mbox{where} \quad n \in \mathbb{Z}.
$$

The resulting function is a periodic function of the frequency variable $\omega$. In the above definition we assume that $\omega$ is expressed in normalised units of radians/sample so that the periodicity is $2\pi$. In terms of the usual time frequency variable $f$, where $\omega = 2\pi f$, we would define it as

函数的结果是以频率变量$\omega$的周期函数，以上定义，我们假设$\omega$在单位圆上，所以周期为$2\pi$。通常的时间频率变量$f$而言,$\omega = 2\pi f$,我们将它定义为

<a id='math:eq:8_002'></a><!--\label{math:eq:8_002}-->$$
Y_{f_s}(f) = \sum_{n\,=\,-\infty}^{\infty} y_n\,e^{-2\pi\imath f t_n},
$$

where $t_n$ is a time coordinate and the subscript $f_s$ denotes the period of $Y_{f_s}(f)$. As we will see in [$\S$ 2.9 &#10142;](../2_Mathematical_Groundwork/2_9_sampling_theory.ipynb) the DTFT (more correctly the DFT introduced below) arises naturally when we take the Fourier transform of a sampled continuous function.

在$t_n$是一个时间坐标和下标$f_s$表示$Y_{f_s}(f)$。正如我们将在2.9中看到的，当我们对一个采样的连续函数进行傅里叶变换时，DTFT(更准确地说是下面介绍的DFT)自然地出现了。

As with the continuous Fourier transform, it is only possible to compute the DTFT analytically in a limited number of cases (eg. when the limit of the infinite series is known analytically or when the signal is band limited i.e. the signal contains only frequencies below a certain threshold). For what follows we will find it useful to review the concept of periodic summation and the Poisson summation formula. Note that the DTFT is defined over the entire field of complex numbers and that there are an infinite number of components involved in the definition.  

与连续傅里叶变换一样，只能在有限的情况下计算DTFT。当分析已知无穷级数的极限时，或当信号的频带有限时，即信号只包含低于某一阈值的频率时)。对于下面的内容，我们将发现回顾周期求和的概念和泊松求和公式是有用的。请注意，DTFT是在复数的整个字段上定义的，定义中涉及无限多个成分。

#### 2.8.1.1 Periodic summation and the DTFT <a id='math:sec:Periodic_summation'></a>

The idea behind periodic summation is to construct a periodic function, $g_{\tau}(t)$ say, from a contnuous function $g(t)$. Consider the following construction 

周期求和背后的理念是构建一个周期函数$g_{\tau}(t)$。考虑以下构造
$$ g_\tau(t) = \sum_{n=-\infty}^{\infty} g(t + n\tau) = \sum_{n=-\infty}^{\infty} g(t - n\tau). $$

Clearly $g_\tau(t)$ has period $\tau$ and looks like an infinite number of copies of the function $g(t)$ for $t$ in the interval $0 \leq t \leq \tau$. We call $g_\tau(t)$ a periodic summation of $g(t)$. Note that we recover $g(t)$ when $n = 0$ and that a similar construction is obviously possible in the frequency domain. Actually the DTFT naturally results in a periodic function of the form

显然$g_\tau(t)$周期为$\tau$,看起来就像一个无限数量的副本,函数$g(t)$的区间$0 \leq t \leq \tau$。我们称之为$g_\tau(t)$的周期和$g(t)$。注意,当$n = 0$我们恢复$g(t)$，在相似的构造在频域中显然是可能的。实际上DTFT很自然地产生了这种形式的周期函数
$$Y_{f_s}(f) = \sum_{k = -\infty}^{\infty} Y(f - k f_s), $$

such that $Y_{f_s}(f)$ is the periodic summation of $Y(f)$. As we will see later, the period $f_s$ is set by the number of samples $N$ at which we have the signal. In [$\S$ 2.9 &#10142;](../2_Mathematical_Groundwork/2_9_sampling_theory.ipynb) we will find it useful to think of $Y(f)$ as the spectrum of a bandlimited signal, $y(t)$ say. When the maximum frequency present in the signal is below a certain threshold the $Y_{f_s}(f)$ with $k \neq 0$ are exact copies of $Y(f)$ which we call aliases. This will become clearer after we have proved the Nyquist-Shannon sampling theorem.  

$Y_{f_s}(f)$是周期性的总和$Y(f)$。稍后我们将看到,周期$f_s$被设置为信号的样本数目$N$。在2.9中我们会发现认为$Y(f)$的频谱带宽是受限的信号，这是有用的。在我们证明了Nyquist-Shannon抽样定理之后，这将变得更加清晰。

#### 2.8.1.2 Poisson summation formula <a id='math:sec:Poisson_summation'></a>

The Poisson summation formula is a result from analysis which is very important in Fourier theory. A general proof of this result will not add much to the current discussion. Instead we will simply point out its implications for Fourier theory as this will result in a particularly transparent proof of the Nyquist-Shannon sampling theorem. 

泊松求和公式是傅里叶理论中非常重要的分析结果。对这一结果的一般性证明不会给当前的讨论增加多少内容。相反，我们将简单地指出它对傅里叶理论的含义，因为这将导致一个特别透明的证明，证明Nyquist-Shannon采样定理。

Basically the Poisson summation formula can be used to relate the Fourier series coefficients of a periodic summation of a function to values which are proportional to the function's continuous Fourier transform. The Poisson summation formula states that, if $Y(f)$ is the Fourier transform of the (Schwartz) function $y(t)$, then

基本上，泊松求和公式可以用来将函数周期求和的傅里叶级数系数与与函数的连续傅里叶变换成正比的值联系起来。泊松求和公式表示：如果$Y(f)$是函数$y(t)$的傅里叶变换(Schwartz)

<a id='math:eq:8_003'></a><!--\label{math:eq:8_003}-->$$
\sum_{n = -\infty}^{\infty} \Delta t ~ y(\Delta t n) e^{-2\pi\imath f \Delta t n} = \sum_{k = -\infty}^{\infty} Y(f - \frac{k}{\Delta t}) = \sum_{k = -\infty}^{\infty} Y(f - kf_s) = Y_{f_s}(f). $$

This shows that the series $y_n = \Delta t y(\Delta t n)$ is sufficient to construct a periodic summation of $Y(f)$. The utility of this construction will become apparent a bit later. For now simply note that it is possible to construct $Y_{f_s}(f)$ as a Fourier series with coefficients $y_n = \Delta t \ y(n\Delta t)$. 

这表明，级数$y_n = \Delta ty (\Delta tn)$足以构造一个$y (f)$的周期和。这种构造的效用将在稍后变得明显。现在只需注意，可以将$y_n = \Delta t \ y(n\Delta t)$构造成系数为$y_n = \Delta t \ y(n\Delta t)$的傅立叶级数。

The above discussion will mainly serve as a theoretical tool. It does not provide an obvious way to perform the Fourier transform in practice because it still requires an infinite number of components $y_n$. Before illustrating its utility we should construct a practical way to implement the Fourier transform.  

以上讨论将主要作为理论工具。它没有提供一个明显的方式进行傅里叶变换在实践中因为它仍然需要一个无限数量的$y_n$。在说明其效用之前，我们应该构造一种实现傅里叶变换的实用方法。

### 2.8.2. The discrete Fourier transform: definition<a id='math:sec:the_discrete_fourier_transform_definition'></a>

Let $y= \left\{y_n \in \mathbb{C}\right\}_{n = 0, \ldots, N-1}$ be a finite set of complex numbers. Then the discrete Fourier transform (DFT) of $y$, denoted $\mathscr{F}_{\rm D}\{y\}$, is defined as

让$y= \left\{y_n \in \mathbb{C}\right\}_{n = 0, \ldots, N-1}$是有一个有限的复数集合。然后将$y$的离散傅里叶变换(DFT)定义为$\mathscr{F}_{\rm D}\{y\}$

<a id='math:eq:8_004'></a><!--\label{math:eq:8_004}-->$$
\mathscr{F}_{\rm D}: \left\{y_n \in \mathbb{C}\right\}_{n \,=\, 0, \ldots, N-1} \rightarrow \left\{Y_k \in \mathbb{C}\right\}_{k \,=\, 0, \ldots, N-1}\\
\mathscr{F}_{\rm D}\{y\} = \left\{Y_k\in\mathbb{C}\right\}_{k \,=\, 0, \ldots, N-1} \quad \mbox{where} \quad 
Y_k = \sum_{n\,=\,0}^{N-1} y_n\,e^{-2\pi\imath f_k t_n} = \sum_{n\,=\,0}^{N-1} y_n\,e^{-\imath 2\pi \frac{nk}{N}}.
$$

In the above definition $f_k$ is the $k$-th frequency sample and $t_n$ is the $n$-th sampling instant. When the samples are spaced at uniform intervals $\Delta t$ apart these are given by

在上面的定义中，$f_k$是$k$-th频率采样，$t_n$是$n$-th采样瞬间。当样本以相同的间隔间隔为$\Delta t$时，由

$$ t_n = t_0 + n\Delta t \quad \mbox{and} \quad f_k = \frac{kf_s}{N} \quad \mbox{where} \quad f_s = \frac{1}{\Delta t}. $$

Most of the proofs shown below are easiest to establish when thinking of the DFT in terms of the actual indices $k$ and $n$. This definition also has the advantage that the samples do not have to be uniformly spaced apart. In this section we use the notation

当考虑DFT时，考虑实际指数$k$和$n$时，下面所示的大多数证明都是最容易建立的。这个定义还有一个优点，那就是样本之间不需要均匀间隔。在本节中，我们使用符号

$$ \mathscr{F}_{\rm D}\{y\}_k = Y_k =  \sum_{n\,=\,0}^{N-1} y_n\,e^{-\imath 2\pi \frac{nk}{N}}, $$

where the subscript $k$ on the LHS denotes the index not involved in the summation. Varaibles such as $Y_k$ and $y_n$ which are related as in the above expression are sometimes refered to as Fourier pairs or Fourier duals.

其中下标$k$表示指数没有参与求和。变量如$Y_k$和$y_n$相关是在上面的表达有时被称作傅里叶对。

The number of Fourier transformed components $Y_k$ is the same as the number of samples of $y_n$. Denoting the set of Fourier transformed components by $Y = \left\{Y_k \in \mathbb{C}\right\}_{k = 0, \ldots, N-1}$, we can define the inverse discrete Fourier transform of $Y$, denoted $\mathscr{F}_{\rm D}^{-1}\{Y\}$, as

傅里叶变换后的分量$Y_k$等于$y_n$的样本值。表示傅里叶变换后的一组分量$Y = \left\{Y_k \in \mathbb{C}\right\}_{k = 0, \ldots, N-1}$,我们可以定义Y的傅里叶反变换$\mathscr{F}_{\rm D}^{-1}\{Y\}$。

<a id='math:eq:8_005'></a><!--\label{math:eq:8_005}-->$$
\mathscr{F}_{\rm D}^{-1}: \left\{Y_k \in \mathbb{C}\right\}_{k \,=\, 0, \ldots, N-1} \rightarrow \left\{y_n \in \mathbb{C}\right\}_{n \,=\, 0, \ldots, N-1}\\
\mathscr{F}_{\rm D}^{-1}\{Y\} = \left\{y_n\in\mathbb{C}\right\}_{n = 0, \ldots, N-1}
\quad \mbox{where} \quad y_n = \frac{1}{N} \sum_{k \ = \ 0}^{N-1} Y_k e^{\imath 2\pi \frac{nk}{N}} \ ,
$$

or in the abbreviated notation

或者用简写

$$ \mathscr{F}_{\rm D}^{-1}\{Y\}_n = y_n = \frac{1}{N} \sum_{k\,=\,0}^{N-1} Y_k\,e^{\imath 2\pi \frac{nk}{N}}. $$

The factor of $\frac{1}{N}$ appearing in the definition of the inverse DFT is a normalisation factor. Some texts even omit it completely. We will follow the above convention throughout the course. The inverse DFT is the inverse operation with respect to the discrete Fourier transform (restricted to the original domain). This can be shown as follows:<br><br>

出现在傅里叶逆变换的定义中的因子$\frac{1}{N}$是一个常数。有些文本甚至完全省略了它。我们将在整个课程中遵循上述惯例。逆DFT是关于离散傅里叶变换的逆运算(仅限于原域)。这可以如下所示

<a id='math:eq:8_006'></a><!--\label{math:eq:8_006}-->$$
\begin{align}
\mathscr{F}_{\rm D}^{-1}\left\{\mathscr{F}_{\rm D}\left\{y\right\}\right\}_{n^\prime} \,&=\, \frac{1}{N}\sum_{k\,=\,0}^{N-1} \left(\sum_{n\,=\,0}^{N-1} y_n e^{-\imath 2\pi\frac{kn}{N}}\right)e^{\imath 2\pi\frac{kn^\prime}{N}}\\
&=\,\frac{1}{N}\sum_{k\,=\,0}^{N-1} \sum_{n\,=\,0}^{N-1} \left( y_n e^{-\imath 2\pi\frac{kn}{N}}e^{\imath 2\pi\frac{kn^\prime}{N}}\right)\\
&=\,\frac{1}{N}\left(\sum_{k\,=\,0}^{N-1} y_{n^\prime}+\sum_{\begin{split}n\,&=\,0\\n\,&\neq\,n^\prime\end{split}}^{N-1} \sum_{k\,=\,0}^{N-1} y_n e^{-\imath 2\pi\frac{kn}{N}}e^{\imath 2\pi\frac{kn^\prime}{N}}\right)\\
&=\,\frac{1}{N}\left(\sum_{k\,=\,0}^{N-1} y_{n^\prime}+\sum_{\begin{split}n\,&=\,0\\n\,&\neq\,n^\prime\end{split}}^{N-1} \sum_{k\,=\,0}^{N-1} y_n e^{\imath 2\pi\frac{k(n^\prime-n)}{N}}\right)\\
&=\,y_{n^\prime}+\frac{1}{N}\sum_{\begin{split}n\,&=\,0\\n\,&\neq\,n^\prime\end{split}}^{N-1} y_n \sum_{k\,=\,0}^{N-1} \left(e^{\imath 2\pi\frac{(n^\prime-n)}{N}}\right)^k\\
&=\,y_{n^\prime}+\frac{1}{N}\sum_{\begin{split}n\,&=\,0\\n\,&\neq\,n^\prime\end{split}}^{N-1} y_n \frac{1-\left(e^{\imath 2\pi\frac{(n^\prime-n)}{N}}\right)^N}{1-\left(e^{\imath 2\pi\frac{(n^\prime-n)}{N}}\right)}\\
&=\,y_{n^\prime}+\frac{1}{N}\sum_{\begin{split}n\,&=\,0\\n\,&\neq\,n^\prime\end{split}}^{N-1} y_n \frac{1-e^{\imath 2\pi(n^\prime-n)}}{1-e^{\imath 2\pi\frac{(n^\prime-n)}{N}}}\\
&\underset{n,n^\prime \in \mathbb{N}}{=}\,y_{n^\prime},\\
\end{align}
$$

where we made use of the identity $\sum_{n\,=\,0}^{N-1}x^n \,=\, \frac{1-x^N}{1-x}$ and used the orthogonality of the sinusoids in the last step. 

其中我们使用公式$\sum_{n\,=\,0}^{N-1}x^n \,=\, \frac{1-x^N}{1-x}$并且在最后一步采用正弦函数作为正交基

Clearly both the DFT and its inverse are periodic with period $N$

显然DFT和逆DFT都是以$N$为周期

<a id='math:eq:8_007'></a><!--\label{math:eq:8_007}-->$$
\begin{align}
\mathscr{F}_{\rm D}\{y \}_k \,&=\,\mathscr{F}_{\rm D}\{y \}_{k \pm N}  \\
\mathscr{F}_{\rm D}^{-1}\{Y \}_{n} \,&=\,\mathscr{F}_{\rm D}^{-1}\{Y \}_{n \pm N}.\\
\end{align}
$$

As is the case for the continuous Fourier transform, the inverse DFT can be expressed in terms of the forward DFT (without proof, but it's straightforward)

与连续傅里叶变换一样，逆DFT可以用正DFT表示(没有证明，但很简单)

<a id='math:eq:8_008'></a><!--\label{math:eq:8_008}-->$$
\begin{align}
\mathscr{F}_{\rm D}^{-1}\{Y\}_n \,&=\, \frac{1}{N} \mathscr{F}_{\rm D}\{Y\}_{-n} \\
&=\,\frac{1}{N} \mathscr{F}_{\rm D}\{Y\}_{N-n}.\\
\end{align}
$$

The DFT of a real-valued set of numbers $y = \left\{y_n \in \mathbb{R}\right\}_{n\,=\,0, \ldots, \,N-1}$ is Hermitian (and vice versa)

一系列实值$y = \left\{y_n \in \mathbb{R}\right\}_{n\,=\,0, \ldots, \,N-1}$的DFT是满足Hermitian性质的（反之亦然）

<a id='math:eq:8_009'></a><!--\label{math:eq:8_009}-->$$
\begin{split}
\mathscr{F}_{\rm D}\{y\}_k\,&=\, \left(\mathscr{F}_{\rm D}\{y\}_{-k}\right)^*\\
&=\, \left(\mathscr{F}_{\rm D}\{y\}_{N-k}\right)^* \ .
\end{split}
$$

### 2.8.3. The Discrete convolution: definition and discrete convolution theorem   离散卷积:定义和离散卷积定理<a id='math:sec:the_discrete_convolution_definition_and_discrete_convolution_theorem'></a>

For two sets of complex numbers $y = \left\{y_n \in \mathbb{C}\right\}_{n = 0, \ldots, N-1}$ and $z = \left\{z_n \in \mathbb{C}\right\}_{n = 0, \ldots, N-1}$ the discrete convolution is, in analogy to the analytic convolution, defined as

与解析卷积类似，对于两系列数$y = \left\{y_n \in \mathbb{C}\right\}_{n = 0, \ldots, N-1}$ 和 $z = \left\{z_n \in \mathbb{C}\right\}_{n = 0, \ldots, N-1}$，离散卷积定义如下

<a id='math:eq:8_010'></a><!--\label{math:eq:8_010}-->$$
\circ: \left\{y_n \in \mathbb{C}\right\}_{n \,=\, 0, \ldots, N-1}\times \left\{z_n \in \mathbb{C}\right\}_{n \,=\, 0, \ldots, N-1} \rightarrow \left\{r_k \in \mathbb{C}\right\}_{k \,=\,  0, \ldots, N-1}\\
(y\circ z)_k = r_k = \sum_{n\,=\,0}^{N-1} y_n z_{k-n}.\\
$$

However there is a bit of a subtlety in this definition. We have to take into account that if $n > k$ the index $k-n$ will be negative. Since we have defined our indices as being strictly positive, this requires introducing what is sometimes referred to as the "wraparound" convention. Recal that complex numbers $r_k = e^{\frac{\imath 2\pi k}{N}}$ have the property that $r_{k \pm mN} = r_k$, where $m \in \mathbb{Z}$ is an integer. In the "wraparound" convention we map indices lying outside the range $0, \cdots , N-1$ into this range using the modulo operator. In other words we amend the definition as follows

然而，这个定义有点微妙。我们必须考虑,如果$n > k$;索引$k-n$将是负的。由于我们将指数定义为严格正的，这就需要引入有时被称为“概括”的约定。回想复数$r_k = e^{\frac{\imath 2\pi k}{N}}$有这一性质：$r_{k \pm mN} = r_k$, 其中 $m \in \mathbb{Z}$是个整数

$$ (y\circ z)_k = r_k = \sum_{n\,=\,0}^{N-1} y_n z_{(k-n) \, \text{mod} \, N}, $$

where mod denotes the modulo operation. Just like the ordinary convolution, the discrete convolution is commutative. One important effect evident from this equation is that if the two series are "broad" enough, the convolution will be continued at the beginning of the series, an effect called aliasing.

其中mod表示模运算。就像普通卷积一样，离散卷积是可交换的。从这个方程中可以明显看出一个重要的效果，如果两个级数足够“宽”，卷积将在级数开始时继续进行，这一效果称为混叠。

The convolution theorem (i.e. that convolution in one domain is the pointwise product in the other domain) is also valid for the DFT and the discrete convolution operator. We state the theorem here without proof (it is similar to the proof for the continuous case). Let $(y \odot z)_n \underset{def}{=} y_n ~ z_n$ (this is the Hadamard or component-wise product, we will encounter it again in [$\S$ 2.10 &#10142;](../2_Mathematical_Groundwork/2_10_linear_algebra.ipynb)). Then, for Fourier pairs $Y_k$ and $y_n$, and $Z_k$ and $z_n$, we have 

卷积定理(即一个域中的卷积是另一个域中的点积)对于DFT和离散卷积算子也是有效的。我们在这里陈述的定理没有证明(它类似于连续情况下的证明)。定义：$(y \odot z)_n \underset{def}{=} y_n ~ z_n$（这是Hadamard或元素逐位的乘积，我们将在2.10节再次遇到它）。对于傅里叶对$Y_k$ 和 $y_n$， $Z_k$ 和 $z_n$，我们有

<a id='math:eq:8_011'></a><!--\label{math:eq:8_011}-->$$
\forall N\,\in\, \mathbb{N}\\
\begin{align}
y \,&=\, \left\{y_n \in \mathbb{C}\right\}_{n\,=\,0, \ldots, \,N-1}\\
z \,&=\, \left\{z_n \in \mathbb{C}\right\}_{n\,=\,0, \ldots, \,N-1}\\
Y \,&=\, \left\{Y_k \in \mathbb{C}\right\}_{k\,=\,0, \ldots, \,N-1}\\
Z \,&=\, \left\{Z_k \in \mathbb{C}\right\}_{k\,=\,0, \ldots, \,N-1}\\
\end{align}\\
\begin{split}
\mathscr{F}_{\rm D}\{y\odot z\}\,&=\,\frac{1}{N}\mathscr{F}_{\rm D}\{y\}\circ \mathscr{F}_{\rm D}\{z\}\\
\mathscr{F}_{\rm D}^{-1}\{Y\odot Z\}\,&=\,\mathscr{F}_{\rm D}\{Y\}\circ \mathscr{F}_{\rm D}\{Z\}\\
\mathscr{F}_{\rm D}\{y\circ z\}\,&=\,\mathscr{F}_{\rm D}\{y\} \odot \mathscr{F}_{\rm D}\{z\}\\
\mathscr{F}_{\rm D}^{-1}\{Y\circ Z\}\,&=\,\frac{1}{N}\mathscr{F}_{\rm D}\{Y\} \odot \mathscr{F}_{\rm D}\{Z\}\\
\end{split}
$$

### 2.8.4.Numerically implementing the DFT <a id='math:sec:numerical_DFT'></a>

We now turn to how the DFT is implemented numerically. The most direct way to do this is to sum the components in a double loop of the form

现在我们来看看DFT是如何在数字上实现的。最直接的方法是在表单的双循环中对组件求和

In [ ]:
def loop_DFT(x):
    """
    Implementing the DFT in a double loop
    Input: x = the vector we want to find the DFT of
    """
    #Get the length of the vector (will only work for 1D arrays)
    N = x.size
    #Create vector to store result in
    X = np.zeros(N,dtype=complex)
    for k in range(N):
        for n in range(N):
            X[k] += np.exp(-1j*2.0*np.pi*k*n/N)*x[n]
    return X

Althought this would produce the correct result, this way of implementing the DFT is going to be incredibly slow. The DFT can be implemented in matrix form. Convince yourself that a vectorised implementation of this operation can be achieved with

虽然这将产生正确的结果，但这种实现DFT的方法将非常慢。DFT可以用矩阵形式实现。说服自己，这个操作可以用向量化实现


$$ X = K x $$
where $K$ is the kernel matrix, it stores the values $K_{kn} = e^{\frac{-\imath 2 \pi k n}{N}}$. This is implemented numerically as follows

其中$K$是矩阵核，它存储的值为 $K_{kn} = e^{\frac{-\imath 2 \pi k n}{N}}$。它的数值实现如下

In [ ]:
def matrix_DFT(x):
    """
    Implementing the DFT in vectorised form
    Input: x = the vector we want to find the DFT of
    """
    #Get the length of the vector (will only work for 1D arrays)
    N = x.size
    #Create vector to store result in
    n = np.arange(N)
    k = n.reshape((N,1))
    K = np.exp(-1j*2.0*np.pi*k*n/N)
    return K.dot(x)

This function will be much faster than the previous implementation. We should check that they both return the same result.

这个函数将比以前的实现快得多。我们应该检查它们是否返回相同的结果

In [ ]:
x = np.random.random(256)  #create random vector to take the DFT of
np.allclose(loop_DFT(x),matrix_DFT(x)) #compare the result using numpy's built in function

Just to be sure our DFT really works, let's also compare the output of our function to numpy's built in DFT function (note numpy automatically implements a faster version of the DFT called the FFT, see the discussion below)

了确保DFT确实有效，我们还将函数的输出与内置在DFT函数中的numpy的输出进行比较(注意，numpy自动实现了一个名为FFT的DFT的更快版本，参见下面的讨论)

In [ ]:
x = np.random.random(256)  #create random vector to take the DFT of
np.allclose(np.fft.fft(x),matrix_DFT(x)) #compare the result using numpy's built in function

Great! Our function is returning the correct result. Next we do an example to demonstrate the duality between the spectral (frequency domain) and temporal (time domain) representations of a function. As the following example shows, the Fourier transform of a time series returns the frequencies contained in the signal.   

太棒了!我们的函数返回正确的结果。接下来，我们做一个例子来演示函数的频谱(频域)表示和时间(时域)表示之间的对偶性。如下例所示，时间序列的傅里叶变换返回信号中包含的频率。

The following code simulates a signal of the form

下面的代码模拟信号

$$ y = \sin(2\pi f_1 t) + \sin(2\pi f_2 t) + \sin(2\pi f_3 t), $$

takes the DFT and plots the amplitude and phase of the resulting components $Y_k$. 

采用DFT和画出信号的振幅和相位

In [ ]:
#First we simulate a time series as the sum of a number of sinusoids each with a different frequency
N = 512  #The number of samples of the time series
tmin = -10 #The minimum value of the time coordinate
tmax = 10 #The maximum value of the time coordinate
t = np.linspace(tmin,tmax,N) #The time coordinate
f1 = 1.0 #The frequency of the first sinusoid
f2 = 2.0 #The frequency of the second sinusoid
f3 = 3.0 #The frequency of the third sinusoid
#Generate the signal
y = np.sin(2.0*np.pi*f1*t) + np.sin(2.0*np.pi*f2*t) + np.sin(2.0*np.pi*f3*t)
#Take the DFT
Y = matrix_DFT(y)
#Plot the absolute value, real and imaginary parts
plt.figure(figsize=(15, 6))
plt.subplot(121)
plt.stem(abs(Y))
plt.xlabel('$k$',fontsize=18)
plt.ylabel(r'$|Y_k|$',fontsize=18)
plt.subplot(122)
plt.stem(np.angle(Y))
plt.xlabel('$k$',fontsize=18)
plt.ylabel(r'phase$(Y_k)$',fontsize=18)

**Figure 2.8.1:** Amplitude and phase plots of the fourier transform of a signal comprised of 3 different tones

由三个不同音调组成的信号的傅里叶变换的幅值和相位图

It is not immediately obvious that these are the frequencies contained in the signal. However, recall, from the definition given at the outset, that the frequencies are related to the index $k$ via

这些频率是否包含在信号中并不明显。然而,从一开始就给出的定义,相关频率指数$k$通过
$$ f_k = \frac{k f_s}{N}, $$
where $f_s$ is the sampling frequency (i.e. one divided by the sampling period). Let's see what happens if we plot the $X_k$ against the $f_k$ using the following bit of code

其中$f_s$是采样频率(即一个除以采样周期)。让我们看看会发生什么,如果我们画出$X_k$相对于$f_k$使用以下代码

In [ ]:
#Get the sampling frequency
delt = t[1] - t[0]
fs = 1.0/delt
k = np.arange(N)
fk = k*fs/N
plt.figure(figsize=(15, 6))
plt.subplot(121)
plt.stem(fk,abs(Y))
plt.xlabel('$f_k$',fontsize=18)
plt.ylabel(r'$|Y_k|$',fontsize=18)
plt.subplot(122)
plt.stem(fk,np.angle(Y))
plt.xlabel('$f_k$',fontsize=18)
plt.ylabel(r'phase$(Y_k)$',fontsize=18)

**Figure 2.8.2:** The fourier transformed signal labled by frequency

Here we see that the three main peaks correspond to the frequencies contained in the input signal viz. $f_1 = 1$Hz, $f_2 = 2$Hz and $f_3 = 3$Hz. But what do the other peaks mean? The additional frequency peaks are a consequence of the following facts:
这里我们看到三个主峰对应于输入信号中包含的频率：$f_1 = 1$Hz, $f_2 = 2$Hz and $f_3 = 3$Hz. 但其他峰值意味着什么呢？

* the DFT of a real valued signal is Hermitian (see [Hermitian property of real valued signals &#10549;](#math:eq:8_009)<!--\ref{math:eq:8_009}-->) so that $Y_{-k} = Y_k^*$, 
* the DFT is periodic with period $N$ (see [Periodicity of the DFT &#10549;](#math:eq:8_007)<!--\ref{math:eq:8_007}-->) so that $Y_{k} = Y_{k+N}$. <br>

When used together the above facts imply that $Y_{N-k} = Y_k^*$. This will be important in [$\S$ 2.9 &#10142;](../2_Mathematical_Groundwork/2_9_sampling_theory.ipynb) when we discuss aliasing. Note that these additional frequency peaks contain no new information. For this reason it is only necessary to store the first $\frac{N}{2} + 1$ samples when taking the DFT of a real valued signal.

一起使用时,上述事实暗示$Y_{N-k} = Y_k^*$。在2.9中，当我们讨论混叠时，这一点很重要。注意，这些额外的频率峰值不包含任何新信息。因此只需要存储第一个$\frac{N}{2} + 1$的DFT实值信号。

We have not explained some of the features of the signal viz.

我们还没有解释信号的一些特性，即

   * Why are there non-zero components of $Y_k$ at frequencies that are not present in the input signal?
   
   为什么在输入信号中不存在频率为$Y_k$的非零分量
   * Why do the three main peaks not contain the same amount of power? This is a bit unexpected since all three components of the input signal have the same amplitude.
   
   为什么三个主峰的功率不相等

As we will see in [$\S$ 2.9 &#10142;](../2_Mathematical_Groundwork/2_9_sampling_theory.ipynb), these features result from the imperfect sampling of the signal. This is unavoidable in any practical application involving the DFT and will be a reoccurring theme throughout this course. You are encouraged to play with the parameters (eg. the minimum $t_{min}$ and maximum $t_{max}$ values of the time coordinate, the number of samples $N$ (do not use $N > 10^5$ points or you might be here for a while), the frequencies of the input components etc.) to get a feel for what does and does not work. In particular try setting the number of samples to $N = 32$ and see if you can explain the output. It might also be a good exercise to and implement the inverse DFT.

正如[$\S$ 2.9 &#10142;](../2_Mathematical_Groundwork/2_9_sampling_theory.ipynb)所看到的，这些特征是由于信号采样不完全造成的。这在涉及DFT的任何实际应用中都是不可避免的，并且将是贯穿本课程的一个反复出现的主题。我们鼓励您使用这些参数(例如。时间坐标的最小值$t_{min}$和最大值$t_{max}$，样本数$N$(不要使用$N > 10^5$点数，否则您可能会在这里待上一段时间)，输入组件的频率等，以了解什么可以工作，什么不可以工作。特别要尝试将示例的数目设置为$N = 32$，看看是否可以解释输出。它也可能是一个很好的练习和实现逆DFT。

We already mentioned that the vectorised version of the DFT above will be much faster than the loop version. We can see exactly how much faster with the following commands 

In [ ]:
%timeit loop_DFT(x)
%timeit matrix_DFT(x)

That is almost a factor of ten difference. Lets compare this to numpy's built in FFT

这几乎是十倍的差别。让我们将其与用FFT构建的numpy进行比较

In [ ]:
%timeit np.fft.fft(x)

That seems amazing! The numpy FFT is about 1000 times faster than our vectorised implementation. But how does numpy achieve this speed up? Well, by using the fast Fourier transform of course.

这似乎令人惊叹!numpy FFT大约比我们的向量化实现快1000倍。但是numpy是如何实现这种加速的呢

### 2.8.5. Fast Fourier transforms<a id='math:sec:fast_fourier_tranforms'></a>

The DFT is a computationally expensive operation. As evidenced by the double loop required to implement the DFT the computational complexity of a naive implementation such as ours scales like $\mathcal{O}(N^2)$ where $N$ is the number of data points. Even a vectorised version of the DFT will scale like $\mathcal{O}(N^2)$ since, in the end, there are still the same number of complex exponentiations and multiplications involved. 

DFT是一种计算开销很大的操作。如实现DFT所需的双循环所示，像我们这样的简单实现的计算复杂度按$\mathcal{O}(N^2)$的比例缩放，其中$N$是数据点的数量。即使是DFT的向量化版本也会像$\mathcal{O}(N^2)$那样伸缩，因为最终涉及的复指数和乘法的数量仍然相同。

By exploiting the symmetries of the DFT, it is not difficult to identify potential ways to safe computing time. Looking at the definition of the discrete Fourier transform [discrete Fourier transform &#10549;](#math:eq:8_004)<!--\ref{math:eq:8_004}-->, one can see that, under certain circumstances, the same summands occur multiple times. Recall that the DFT is periodic i.e. $Y_k = Y_{N+k}$, where $N$ is the number of data points. Now suppose that $N = 8$. In calculating the component $Y_2$ we would have to compute the quantity $y_2\,e^{-2{\pi}\imath\frac{2 \cdot 2}{8}}$ i.e. when $n = 2$. However, using the periodicity of the kernel $e^{-2\pi\imath \frac{kn}{N}} = e^{-2\pi\imath \frac{k(n+N)}{N}}$, we can see that this same quantity will also have to be computed when calculating the component $Y_6$ since $y_2\,e^{-2{\pi}\imath\frac{2\cdot2}{8}}=y_2e^{-2{\pi}\imath\frac{6\cdot2}{8}} = y_2e^{-2{\pi}\imath\frac{12}{8}}$. If we were calculating the DFT by hand, it would be a waste of time to calculate this summand twice. To see how we can exploit this, lets first split the DFT into its odd and even $n$ indices as follows
\begin{eqnarray}
Y_{k} &=& \sum_{n = 0}^{N-1} y_n e^{-2\pi\imath \frac{kn}{N}}\\
 &=& \sum_{m = 0}^{N/2-1} y_{2m} e^{-2\pi\imath \frac{k(2m)}{N}} + \sum_{m = 0}^{N/2-1} y_{2m+1} e^{-2\pi\imath \frac{k(2m+1)}{N}}\\
 &=& \sum_{m = 0}^{N/2-1} y_{2m} e^{-2\pi\imath \frac{km}{N/2}} + e^{-2\pi\imath \frac{k}{N}}\sum_{m = 0}^{N/2-1} y_{2m+1} e^{-2\pi\imath \frac{km}{N/2}}
\end{eqnarray}
Notice that we have split the DFT into two terms which look very much like DFT's of length $N/2$, only with a slight adjustment on the indices. Importantly the form of the kernel (i.e. $e^{-2\pi\imath \frac{km}{N/2}}$) looks the same for both the odd and the even $n$ indices. Now, while $k$ is in the range $0, \cdots , N-1$, $n$ only ranges through $0,\cdots,N/2 - 1$. The DFT written in the above form will therefore be periodic with period $N/2$ and we can exploit this periodic property to compute the DFT with half the number of computations. See the code below for an explicit implementation.

利用离散傅里叶变换的对称性，识别安全计算时间的潜在方法并不困难。看看离散傅里叶变换的定义[discrete Fourier transform &#10549;](#math:eq:8_004)<!--\ref{math:eq:8_004}-->, 可以看出，在某些情况下，相同的和式会出现多次。还记得DFT是周期性的吗?$Y_k = Y_{N+k}$,其中$N$是数据点数。现在假设$N = 8$。在计算$Y_2$的分量时，我们需要的计算数量$y_2\,e^{-2{\pi}\imath\frac{2 \cdot 2}{8}}$ i.e. 当 $n = 2$. 但是，利用核函数$e^{-2\pi\imath \frac{kn}{N}} = e^{-2\pi\imath \frac{k(n+N)}{N}}$的周期性，我们可以看到，当计算分量$Y_6$时，同样的量也需要计算。因为 $y_2\,e^{-2{\pi}\imath\frac{2\cdot2}{8}}=y_2e^{-2{\pi}\imath\frac{6\cdot2}{8}} = y_2e^{-2{\pi}\imath\frac{12}{8}}$。如果我们要手工计算DFT，那么两次计算这个和将是浪费时间。为了了解如何利用这一点，让我们首先将DFT分解为奇数和偶数$n$的指数，如下所示
\begin{eqnarray}
Y_{k} &=& \sum_{n = 0}^{N-1} y_n e^{-2\pi\imath \frac{kn}{N}}\\
 &=& \sum_{m = 0}^{N/2-1} y_{2m} e^{-2\pi\imath \frac{k(2m)}{N}} + \sum_{m = 0}^{N/2-1} y_{2m+1} e^{-2\pi\imath \frac{k(2m+1)}{N}}\\
 &=& \sum_{m = 0}^{N/2-1} y_{2m} e^{-2\pi\imath \frac{km}{N/2}} + e^{-2\pi\imath \frac{k}{N}}\sum_{m = 0}^{N/2-1} y_{2m+1} e^{-2\pi\imath \frac{km}{N/2}}
\end{eqnarray}

注意，我们将DFT分成了两项，这两项看起来非常像长度为$N/2$的DFT，只是对索引进行了轻微调整。重要的是，对于奇数和偶数的$ N $索引，内核的形式(即$e^{-2\pi\imath \frac{km}{N/2}}$)看起来是相同的。现在，当$k$在$0范围内时，\cdots, n -1$， $n$只在$0范围内，\cdots, n /2 -1$。因此，上面形式所写的DFT将是周期为$N/2$的周期函数，我们可以利用这个周期特性用一半的计算量来计算DFT。有关显式实现，请参见下面的代码。

In [ ]:
def one_layer_FFT(x):
    """An implementation of the 1D Cooley-Tukey FFT using one layer"""
    N = x.size
    if N%2>0:
        print "Warning: length of x in not a power of two, returning DFT"
        return matrix_DFT(x)
    else:
        X_even = matrix_DFT(x[::2])
        X_odd = matrix_DFT(x[1::2])
        factor = np.exp(-2j * np.pi * np.arange(N) / N)
        return np.concatenate([X_even + factor[:N / 2] * X_odd,X_even + factor[N / 2:] * X_odd])

Lets confirm that this function returns the correct result by comparing fith numpy's FFT.

让我们通过比较numpy的FFT来确认这个函数返回正确的结果。

In [ ]:
np.allclose(np.fft.fft(x),one_layer_FFT(x))

And voila! We can compute the DFT with one quarter the usual number of operations effectively reducing the computational cost to $\mathcal{O}(N/4)$. This idea was introduced by Cooley und Tukey in 1965 when they published "The Fast Fourier Transform" algorithm. Their algorithm actually splits each smaller DFT up even further, until the arrays are small enough so that the strategy is no longer beneficial. The Cooley and Tukey algorithm has a computational complexity which scales as $\mathcal{O}(N\log_2N)$. An example of a recursive Python implementation (as well as a vectorised one) using their idea can be found here [https://jakevdp.github.io/blog/2013/08/28/understanding-the-fft/  &#10142;](https://jakevdp.github.io/blog/2013/08/28/understanding-the-fft/). The FFT algorithm made it possible to, for the first time, compute the Fourier transform of very large data sets. The FFT is used in so many real life applications that is is probably safe to say it is the single most useful algorithm ever invented.  

我们可以用通常操作数的四分之一来计算DFT，有效地降低了计算成本至$\mathcal{O}(N/4)$。这个想法是由Cooley und Tukey在1965年提出的，当时他们发表了“快速傅里叶变换”算法。他们的算法实际上把每个更小的DFT分割得更大，直到数组足够小，使得策略不再有用。Cooley和Tukey算法具有计算复杂度$\mathcal{O}(N\log_2N)$。可以找到一个使用它们的思想的递归Python实现的例子(以及一个向量化的Python实现)。FFT算法使得第一次计算大型数据集的傅里叶变换成为可能。FFT在许多实际应用中都得到了应用，可以说它是有史以来最实用的算法。

Today's implementations of DFTs make use of many other clever tricks for further optimisation. So much work has gone into efficiently implememting the DFT that it is impossible to descibe every aspect here. Often we can simply use these algorithms as a black-box without really needing to know what is going on behind the scenes. As we will see in the coming chapters, there are at least some details which we should be aware of. For example, in the above implementation the number of data points has to be a power of two to make full use of the computational speed up provided by the FFT. One aspect that we have conveniently left out is the implicit assumption that the data points are spaced equally far apart in time (or frequency). Although this is not really necessary for the definition of the DFT, it does make things a lot simpler. This is one of the crucial requirements of the FFT however and we will see it creep up again later. 

今天使用了许多其他聪明的技巧来进一步优化和实现DFTs。为了有效地实现DFT，已经进行了大量的工作，因此不可能在这里描述每个方面。通常，我们可以简单地将这些算法用作黑匣子，而不需要真正了解幕后发生了什么。我们将在接下来的章节中看到，至少有一些细节我们应该知道。例如，在上面的实现中，为了充分利用FFT提供的计算速度，数据点的数量必须是2的幂。我们很方便地忽略了一个方面，即隐含的假设，即数据点在时间(或频率)上的间隔相等。虽然对于DFT的定义来说，这并不是必需的，但是它确实使事情变得简单了许多。然而，这是FFT的关键要求之一，稍后我们将看到它再次出现

***

* Next: [2.9 Sampling Theory](2_9_sampling_theory.ipynb)